In [3]:
%%capture
%pip install gurobipy
import warnings
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import Model
from gurobipy import GRB
warnings.filterwarnings('ignore')

In [4]:
# Question 1

# Data from the problem
commodities = ["A", "B", "C", "D"]
silos = [1, 2, 3, 4, 5, 6, 7]

# Amount to be stored for each commodity
demand = {"A": 75, "B": 50, "C": 25, "D": 80}

# Capacity of each silo
capacity = {1: 25, 2: 25, 3: 40, 4: 60, 5: 80, 6: 100, 7: 100}

# Fixed cost of using each silo
fixed_cost = {1: 200, 2: 200, 3: 350, 4: 400, 5: 500, 6: 700, 7: 700}

# Variable loading costs (commodity, silo)
loading_cost = {
    "A": {1: 1, 2: 2, 3: 2, 4: 3, 5: 4, 6: 5, 7: 5},
    "B": {1: 2, 2: 3, 3: 3, 4: 3, 5: 1, 6: 5, 7: 5},
    "C": {1: 4, 2: 4, 3: 3, 4: 2, 5: 1, 6: 5, 7: 5},
    "D": {1: 1, 2: 1, 3: 2, 4: 3, 5: 5, 6: 5, 7: 5},
}

# Create the model
model = Model("Grain Storage Optimization")

# Decision Variables
x = {}  # x[i, j] = amount of commodity i stored in silo j
y = {}  # y[j] = 1 if silo j is used, 0 otherwise

for i in commodities:
    for j in silos:
        x[i, j] = model.addVar(vtype=GRB.CONTINUOUS, name=f"x_{i}_{j}")

for j in silos:
    y[j] = model.addVar(vtype=GRB.BINARY, name=f"y_{j}")

# Objective Function: Minimize total cost
model.setObjective(
    sum(loading_cost[i][j] * x[i, j] for i in commodities for j in silos) +
    sum(fixed_cost[j] * y[j] for j in silos),
    GRB.MINIMIZE
)

# Constraints

# 1. Each commodity must be fully stored
for i in commodities:
    model.addConstr(sum(x[i, j] for j in silos) == demand[i], f"storage_{i}")

# 2. Silo capacity constraint
for j in silos:
    model.addConstr(sum(x[i, j] for i in commodities) <= capacity[j], f"capacity_{j}")

# 3. Silo activation constraint
for i in commodities:
    for j in silos:
        model.addConstr(x[i, j] <= capacity[j] * y[j], f"activation_{i}_{j}")

# Solve the model
model.optimize()

# Output results
if model.status == GRB.OPTIMAL:
    print(f"\nOptimal Cost: {model.objVal}")
    print("\nOptimal Storage Schedule:")
    for i in commodities:
        for j in silos:
            if x[i, j].x > 0:
                print(f"Store {x[i, j].x} tons of {i} in Silo {j}")

    print("\nActivated Silos:")
    for j in silos:
        if y[j].x > 0.5:
            print(f"Silo {j} is used.")
else:
    print("No optimal solution found.")

Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 39 rows, 35 columns and 112 nonzeros
Model fingerprint: 0x0c3fd6ce
Variable types: 28 continuous, 7 integer (7 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 7e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 1e+02]
Presolve removed 14 rows and 0 columns
Presolve time: 0.00s
Presolved: 25 rows, 35 columns, 91 nonzeros
Variable types: 28 continuous, 7 integer (7 binary)
Found heuristic solution: objective 3455.0000000

Root relaxation: objective 2.055000e+03, 15 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth 

In [ ]:
# Question 2

from gurobipy import Model, GRB

# Define data
shows = ["Cheers", "Dynasty", "L.A. Law", "Jake", "Bob Newhart", "News Special",
         "Focus on Science", "Beaches", "Urban Action"]
revenues = [6, 10, 9, 4, 5, 2, 6, 7, 8]  # Advertising revenue in million dollars
public_interest = {2, 5, 6, 8}  # Shows with public interest
violence = {1, 2, 3, 5}  # Shows containing violence
focus_on_science = 6  # "Focus on Science"
jake = 3  # "Jake"
la_law = 2  # "L.A. Law"
urban_action = 8  # "Urban Action"
num_shows = 5

# Create model
model = Model("TV Scheduling")

# Decision variables
x = model.addVars(len(shows), vtype=GRB.BINARY, name="x")
y = model.addVar(vtype=GRB.BINARY, name="y")  # Auxiliary variable for revenue penalty

# Objective: Maximize total advertising revenue
model.setObjective(sum(revenues[i] * x[i] for i in range(len(shows))) - 4 * y, GRB.MAXIMIZE)

# Constraint 1: Schedule exactly 5 shows
model.addConstr(sum(x[i] for i in range(len(shows))) == num_shows, "ShowCount")

# Constraint 2: Public Interest shows >= Violence shows
model.addConstr(sum(x[i] for i in public_interest) >= sum(x[i] for i in violence), "PublicVsViolence")

# Constraint 3: If "Focus on Science" is chosen, either "Jake" or "L.A. Law" must be chosen
model.addConstr(x[focus_on_science] <= x[jake] + x[la_law], "FocusScienceDependency")

# Constraint 4: "Focus on Science" and "Urban Action" cannot both be chosen
model.addConstr(x[focus_on_science] + x[urban_action] <= 1, "MutualExclusion")

# Constraint 5: Limit violent shows to 3, otherwise revenue is penalized
model.addConstr(sum(x[i] for i in violence) <= 3 + y, "ViolencePenalty")

# Optimize model
model.optimize()

# Extract results
selected_shows = [shows[i] for i in range(len(shows)) if x[i].x > 0.5]
max_revenue = model.objVal

# Print results
print("Optimal Schedule:", selected_shows)
print("Maximal Revenue:", max_revenue, "million dollars")

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 5 rows, 10 columns and 23 nonzeros
Model fingerprint: 0xe4d47297
Variable types: 0 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Presolve removed 3 rows and 6 columns
Presolve time: 0.00s
Presolved: 2 rows, 4 columns, 6 nonzeros
Variable types: 0 continuous, 4 integer (4 binary)
Found heuristic solution: objective 40.0000000

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 2 (of 2 available processors)

Solution count 1: 40 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.000000000000e+01, best bound 4.000000000000e+01, gap 0.0000%